In [1]:
from tqdm import tqdm
import os

In [2]:
with open('BM-A-pt3') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    objektif = t.split('objektif: ')[1].split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'objektif': objektif,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [3]:
questions[:3]

[{'no': '1',
  'objektif': 'Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.',
  'soalan': 'Para ___ tanah air telah dihantar ke negara Jepun untuk membantu mangsa gempa bumi dan tsunami.\nA. hartawan\nB. dermawan\nC. bangsawan\nD. sukarelawan',
  'jawapan': 'D'},
 {'no': '2',
  'objektif': 'Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.',
  'soalan': 'Kebanyakan barang yang disimpan di dalam stor itu telah rosak ____ tikus.\nA. digerit\nB. digigit\nC. dikesip\nD. diketip',
  'jawapan': 'A'},
 {'no': '3',
  'objektif': 'Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.',
  'soalan': 'Shafiq cuba mengubah kedudukan meja kayu itu ___ ruang tamu ___ ruang dapur rumahnya.\nA. dari ... ke\nB. dari ... kepada\nC. daripada ... ke\nD. daripada ... kepada',
  'jawapan': 'A'}]

In [4]:
row = questions[0]
prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
"""
print(prompt)


objektif: Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.
soalan: Para ___ tanah air telah dihantar ke negara Jepun untuk membantu mangsa gempa bumi dan tsunami.
A. hartawan
B. dermawan
C. bangsawan
D. sukarelawan



In [5]:
from transformers import AutoTokenizer,AutoModelForCausalLM, BitsAndBytesConfig
import torch
import logging
logging.getLogger("transformers").setLevel(logging.CRITICAL)
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type='nf4',
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
p  = "mesolitica/malaysian-llama2-7b-32k-instructions"
tokenizer = AutoTokenizer.from_pretrained(p)
model = AutoModelForCausalLM.from_pretrained(p,quantization_config=nf4_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
device = torch.device("cuda")
max_tok = 256
correct_idx = []
for i in tqdm(range(len(questions))):
    row = questions[i]
    prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
Jawapan:

""".strip()
    try:
        msg = [
            {"role": "system", "content": "Anda sebagai pakar menjawab soalan yang mahir berbahasa. Sila berfikir dgn bijak dan berhati-hati ketika menjawab. Sila menjawab dengan satu huruf sahaja."},
            {"role": "user", "content": prompt},
        ]
        encodeds = tokenizer.apply_chat_template(msg, return_tensors="pt")
        model_inputs = encodeds.to(device)
        generated_ids = model.generate(model_inputs, max_new_tokens=64, do_sample=False)
        decoded = tokenizer.batch_decode(generated_ids)[0].split('[/INST]')[-1].replace("Jawapan: ","").strip()
        if decoded[0] == row['jawapan'][0]:
            correct_idx.append(i)
        else:
            print(prompt)
            print("___"* 20)
            print(decoded +"\n|||\n-->"+row['jawapan'])
            print("___"* 20)
        # r = curl(prompt)
        # answer = r.json()['choices'][0]['message']['content']
        # questions[i]['output'] = answer
    except Exception as e:
        print(e)


  2%|██▋                                                                                                                                           | 1/54 [00:00<00:23,  2.28it/s]

objektif: Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.
soalan: Para ___ tanah air telah dihantar ke negara Jepun untuk membantu mangsa gempa bumi dan tsunami.
A. hartawan
B. dermawan
C. bangsawan
D. sukarelawan
Jawapan:
____________________________________________________________
A
|||
-->D
____________________________________________________________


  7%|██████████▌                                                                                                                                   | 4/54 [00:02<00:44,  1.12it/s]

objektif: Ayat-ayat di bawah mungkin betul dan mungkin salah. Jika terdapat kesalahan, kesalahan itu telah ditandakan dengan A, B atau C dan jika tiada kesalahan telah ditandakan D.
soalan: Kemunculan budaya K-Pop menyebabkan adat [ketimuran](A) semakin dilupakan oleh [setengah-setengah](B) Generasi Y [pada](C) masa ini.
A. ketimuran
B. setengah-setengah
C. pada
D. tiada kesalahan
Jawapan:
____________________________________________________________
A
|||
-->D
____________________________________________________________


  9%|█████████████▏                                                                                                                                | 5/54 [00:04<00:51,  1.05s/it]

...

objektif: Baca petikan di bawah ini dengan teliti, kemudian jawab soalan-soalan yang berikut.
soalan: Demam denggi merupakan penyakit bawaan nyamuk Aedes yang menyebabkan
virus demam denggi kepada manusia. Kita sering kali alpa dan memandang enteng
tentang wabak denggi sehingga ada dalam kalangan ahli keluarga kita dijangkiti demam
denggi.
Acap kali kita mendengar bahawa pesakit yang mengalami demam denggi bagi kali
kedua beldepan dengan risiko kematian. Hal ini demikian kerana antibodi badan mangsa
sudah lemah dan tidak mampu melawan serangan virus denggi. Virus ini disebarkan oleh
nyamuk aedes betina. Hakikatnya, risiko kematian sentiasa ada bagi setiap pesakit
demam denggi walaupun kalipertama dijangkiti virus berkenaan.
Demam denggi bermuia dengan pesakit mengalami demam panas diikuti dengan
simptom-simptom lain seperti sakit kepala, sakit sendi dan sakit pada bahagian beiakang
mata serta ruam merah di kulit. Apabila platlet darah pesakit demam denggr semakin
berkurangan secara ber

 33%|███████████████████████████████████████████████                                                                                              | 18/54 [00:12<00:22,  1.61it/s]

objektif: Baca petikan di bawah ini dengan teliti, kemudian jawab soalan-soalan yang berikut.
soalan: Demam denggi merupakan penyakit bawaan nyamuk Aedes yang menyebabkan
virus demam denggi kepada manusia. Kita sering kali alpa dan memandang enteng
tentang wabak denggi sehingga ada dalam kalangan ahli keluarga kita dijangkiti demam
denggi.
Acap kali kita mendengar bahawa pesakit yang mengalami demam denggi bagi kali
kedua beldepan dengan risiko kematian. Hal ini demikian kerana antibodi badan mangsa
sudah lemah dan tidak mampu melawan serangan virus denggi. Virus ini disebarkan oleh
nyamuk aedes betina. Hakikatnya, risiko kematian sentiasa ada bagi setiap pesakit
demam denggi walaupun kalipertama dijangkiti virus berkenaan.
Demam denggi bermuia dengan pesakit mengalami demam panas diikuti dengan
simptom-simptom lain seperti sakit kepala, sakit sendi dan sakit pada bahagian beiakang
mata serta ruam merah di kulit. Apabila platlet darah pesakit demam denggr semakin
berkurangan secara ber

 33%|███████████████████████████████████████████████                                                                                              | 18/54 [00:15<00:30,  1.18it/s]


KeyboardInterrupt: 

In [18]:
len(correct_idx)/len(questions) * 100

37.03703703703704

In [19]:
correct_idx

[1, 2, 6, 13, 14, 15, 16, 19, 23, 24, 25, 33, 37, 41, 42, 44, 45, 47, 48, 51]